In [1]:
import pandas as pd
import numpy as np

import json
import requests
import warnings

from web3 import Web3
from datetime import datetime

In [2]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## Setup and load data

In [8]:
# set input variables
BASE_FOLDER = 'data/'
TXNS_FILENAME = 'txns-du.csv'

POOL_ADDRESS = '0xAAB9EaBa1AA2653c1Dda9846334700b9F5e14E44'

OCEAN_TOKEN = '0x967da4048cD07aB37855c090aAF366e4ce1b9F48'

INFURA_API_KEY = '7e9aa77d7cfc4ffcb6718a643d8cccd8'

In [4]:
# get contract abi
ABI_ENDPOINT = 'https://api.etherscan.io/api?module=contract&action=getabi&address=' + POOL_ADDRESS
ABI = json.loads(requests.get(ABI_ENDPOINT).json()['result'])

In [5]:
# define contract
web3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/' + INFURA_API_KEY))
contract = web3.eth.contract(POOL_ADDRESS, abi=ABI)

In [6]:
# contract main variables
TOTAL_SUPPLY = contract.functions.totalSupply().call()
TOTAL_OCEAN = contract.functions.getBalance(OCEAN_TOKEN).call()

TOKEN_LIST = contract.functions.getCurrentTokens().call()
TOKEN_LIST.remove(OCEAN_TOKEN)
TOKENS_DICT = {
    OCEAN_TOKEN: 'OCEAN',
    TOKEN_LIST[0]: 'DATA_TOKEN'
}

In [9]:
# load txns data
df = pd.read_csv(BASE_FOLDER + TXNS_FILENAME, index_col=False)
df.head(5)

,Txhash,Blockno,UnixTimestamp,DateTime,From,To,ContractAddress,Value_IN(ETH),Value_OUT(ETH),CurrentValue @ $4568.42/Eth,TxnFee(ETH),TxnFee(USD),Historical $Price/Eth,Status,ErrCode,Method
0,0x23ad38f4a8dfc0598c659044404e328124835868c8309b61357835fda4526544,11262585,1605445590,2020-11-15 13:06:30,0x655efe6eb2021b8cefe22794d90293aec37bb325,0xaab9eaba1aa2653c1dda9846334700b9f5e14e44,NaN,0,0,0,0.012888,58.877116,448.51,NaN,NaN,Setup
1,0x3e2ecb226ca2f887c608bcb8733853473821dfbb4b4f295096e88141979b22f1,11262588,1605445653,2020-11-15 13:07:33,0xcc7e9b8331bea863a158589e8ebcf118c72d0683,0xaab9eaba1aa2653c1dda9846334700b9f5e14e44,NaN,0,0,0,0.185826,848.931215,448.51,NaN,NaN,Joinswap Extern Amount In
2,0xc19c05f76169f1ca1eecfca4de905efe55c71bc3427055c04bb36306d1dd10e6,11262588,1605445653,2020-11-15 13:07:33,0xb40156f51103ebaa842590ce51dd2cd0a9e83cda,0xaab9eaba1aa2653c1dda9846334700b9f5e14e44,NaN,0,0,0,0.077555,354.304905,448.51,NaN,NaN,Joinswap Extern Amount In
3,0x211bd5bede82cc2f068868d1f903a57464a75f5b8f59bf25c6f8faf3adab4be1,11262588,1605445653,2020-11-15 13:07:33,0x229ec4c57725f4fccae94c28012330ea958c212e,0xaab9eaba1aa2653c1dda9846334700b9f5e14e44,NaN,0,0,0,0.011816,53.979080,448.51,NaN,NaN,Joinswap Extern Amount In
4,0x1885698f6c8bc5a4819a473e1db744805ac517912c7020baec55a2a573e59acf,11262588,1605445653,2020-11-15 13:07:33,0xcf9e8160a07358d3c13637b262da8fb34ff6b4eb,0xaab9eaba1aa2653c1dda9846334700b9f5e14e44,NaN,0,0,0,0.003339,15.255662,448.51,NaN,NaN,Swap Exact Amount In


## Process txns

In [10]:
dfc = df[df['Status'] != 'Error(0)']

txns_dict = {
    'address_from': [], 
    'method': [],
    'tokenIn': [], 
    'tokenAmountIn': [], 
    'tokenOut': [], 
    'tokenAmountOut': [],
    'date': []
}

for i, r in dfc.iterrows():
    receipt = web3.eth.getTransactionReceipt(r.Txhash)
    if r.Method=='Setup' or r.Method=='Joinswap Extern Amount In':
        logs = contract.events.LOG_JOIN().processReceipt(receipt)
    elif r.Method == 'Swap Exact Amount In' or r.Method == 'Swap Exact Amount Out':
        logs = contract.events.LOG_SWAP().processReceipt(receipt)
    elif r.Method=='Exit Pool' or r.Method == 'Exitswap Extern Amount Out' or r.Method=='Exitswap Pool Amount In':
        logs = contract.events.LOG_EXIT().processReceipt(receipt)
    if len(logs)>0:
        for log in logs:
            args = log['args']
            txns_dict['address_from'].append(args.get('caller'))
            txns_dict['method'].append(r.Method)
            txns_dict['tokenIn'].append(args.get('tokenIn'))
            txns_dict['tokenAmountIn'].append(args.get('tokenAmountIn'))
            txns_dict['tokenOut'].append(args.get('tokenOut'))
            txns_dict['tokenAmountOut'].append(args.get('tokenAmountOut'))
            txns_dict['date'].append(r.DateTime)
    else:
        print('Empty logs:')
        print(r.Txhash, r.Method)    

dfx = pd.DataFrame.from_dict(txns_dict)
dfx['tokenIn'] = dfx['tokenIn'].apply(lambda x: TOKENS_DICT.get(x))
dfx['tokenOut'] = dfx['tokenOut'].apply(lambda x: TOKENS_DICT.get(x))

In [11]:
dfx.head()

,address_from,method,tokenIn,tokenAmountIn,tokenOut,tokenAmountOut,date
0,0x655eFe6Eb2021b8CEfE22794d90293aeC37bb325,Setup,DATA_TOKEN,999985714285714300000,None,NaN,2020-11-15 13:06:30
1,0x655eFe6Eb2021b8CEfE22794d90293aeC37bb325,Setup,OCEAN,116665000000000000000000,None,NaN,2020-11-15 13:06:30
2,0xcC7E9b8331bea863a158589E8EBCF118C72d0683,Joinswap Extern Amount In,OCEAN,14522695122782472713619,None,NaN,2020-11-15 13:07:33
3,0xB40156F51103EbaA842590cE51DD2cD0a9E83cDa,Joinswap Extern Amount In,OCEAN,22205606938923103737842,None,NaN,2020-11-15 13:07:33
4,0x229EC4C57725f4fccAe94c28012330EA958C212e,Joinswap Extern Amount In,OCEAN,1900000000000000000000,None,NaN,2020-11-15 13:07:33


In [12]:
dfx['method'].value_counts()

Joinswap Extern Amount In     437
Exitswap Extern Amount Out    250
Swap Exact Amount In          104
Exitswap Pool Amount In       37 
Exit Pool                     26 
Swap Exact Amount Out         4  
Setup                         2  
Approve                       2  
Transfer                      2  
Name: method, dtype: int64

## Compute insights by address

In [13]:
# helpers

today = datetime.today()

def ocean_amount(x):
    # use than groupby addr to compute net_ocean_in
    if x.tokenIn=='OCEAN':
        return x.tokenAmountIn
    elif x.tokenOut=='OCEAN':
        return - x.tokenAmountOut
    else:
        return 0

def ocean_today_in(addr):
    pool_shares = contract.functions.balanceOf(Web3.toChecksumAddress(addr)).call()
    pool_fraction = pool_shares / TOTAL_SUPPLY
    return TOTAL_OCEAN * pool_fraction

def days_in_pool(x):
    if x.is_still_in is True:
        return (today - x.date_first_txn).days
    else:
        return (x.date_last_txn - x.date_first_txn).days

In [14]:
# compute pool contributions and aggregate by address
dfx['ocean_contrib_pool'] = dfx.apply(lambda x: ocean_amount(x), axis=1)

dfxg = dfx[['address_from', 'ocean_contrib_pool', 'date']].groupby('address_from').agg({
    'ocean_contrib_pool': 'sum',
    'date': ['min', 'max']
}).reset_index()
dfxg.columns = ['address_from', 'ocean_contrib_pool', 'date_first_txn', 'date_last_txn']

In [15]:
# add accounting variables
dfxg['ocean_today_in'] = dfxg['address_from'].apply(lambda x: ocean_today_in(x))
dfxg['is_still_in'] = dfxg['ocean_today_in'] > 0
dfxg['virtual_gain'] = dfxg['ocean_today_in'] - dfxg['ocean_contrib_pool']

# add date variables
dfxg['date_first_txn'] = dfxg['date_first_txn'].apply(lambda x: datetime.strptime(x[:10], '%Y-%m-%d'))
dfxg['date_last_txn'] = dfxg['date_last_txn'].apply(lambda x: datetime.strptime(x[:10], '%Y-%m-%d'))
dfxg['days_since_last_txn'] = dfxg['date_last_txn'].apply(lambda x: (today - x).days)
dfxg['days_in_pool'] = dfxg.apply(lambda x: days_in_pool(x), axis=1)

# convert amounts to readable format
dfxg['ocean_contrib_pool'] = dfxg['ocean_contrib_pool'].apply(lambda x: Web3.fromWei(x, 'ether') if np.sign(x)==1 else -Web3.fromWei(-x, 'ether'))
dfxg['ocean_today_in'] = dfxg['ocean_today_in'].apply(lambda x: Web3.fromWei(x, 'ether'))
dfxg['virtual_gain'] = dfxg['virtual_gain'].apply(lambda x: Web3.fromWei(x, 'ether') if np.sign(x)==1 else -Web3.fromWei(-x, 'ether'))

dfxg.sort_values(['virtual_gain'], ascending=False).head()

,address_from,ocean_contrib_pool,date_first_txn,date_last_txn,ocean_today_in,is_still_in,virtual_gain,days_since_last_txn,days_in_pool
103,0x655eFe6Eb2021b8CEfE22794d90293aeC37bb325,148481.54335857896062976,2020-11-15,2021-08-11,190245.801245510354337792,True,41764.257886931393708032,96,365
271,0xeEdAB724C292e6Ab438E789CDBD2eeA1AE90e9FF,-25345.622891204637097984,2020-12-08,2021-01-10,0,False,25345.622891204637097984,309,33
114,0x766337d18E12Df977b5F54516b2333e39E7dcB5a,-15821.876262462786895872,2020-11-15,2020-11-15,0.00000114278227810459521484375,True,15821.876263605569060864,365,365
179,0xB40156F51103EbaA842590cE51DD2cD0a9E83cDa,-11972.0837353836642304,2020-11-15,2020-12-15,1.79380407124820404052734375E-8,True,11972.083735401603268608,335,365
276,0xf3b823fa20ec85b4eaa9082A7E799520550443Ef,-11205.1033508220502016,2020-11-15,2020-12-08,2.806809159222155776,True,11207.910159981272367104,342,365


In [ ]:
# save final data
dfxg[['address_from', 'ocean_amount', 'ocean_today_in', 'is_still_in', 
      'virtual_gain', 'days_since_last_txn', 'days_in_pool']].sort_values(
          ['virtual_gain'], ascending=False).to_csv(
              BASE_FOLDER + 'addresses-du.csv', index=False)